In [1]:
#================================= Start of importing required packages and libraries =========================================#
from __future__ import print_function
%matplotlib inline
import numpy as np
import torchvision
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from sklearn.metrics import *
from matplotlib import pyplot as plt
from dataset import Dataset
from models import CNNMnist, DenseNet
import os
from torch.utils import data
import math
import random
from tqdm import tqdm
import copy
import statistics
from utils import *
from environment_federated import *
from experiment_federated import *

#================================== End of importing required packages and libraries ==========================================#


In [2]:
#=============================== Defining global variables ========================#
SIM_NUM = 3 # number of simulation times
PEER_NUM = 100 # "number of peers: K" 
PEER_PSEUDONYM = ['Worker '+str(i+1) for i in range(PEER_NUM)]
PEER_FRAC = 0.15 #'the fraction of peers: C to bel selected in each round'
UNTARGETED_ATTACK_RATES = [0.4] #different rates of random attack
STEALTHY_POSONING_RATES = [0.4] #different rates of stealthy attack
RANDOM_MALICIOU_BEHAVIOUR_RATE = 1
STEALTHY_MALICIOU_BEHAVIOUR_RATE = 1
MAPPING_LIST = [0, 1, 2, 3, 4, 5, 6, 7, 8, 4]
SEED = 7 #fixed seed
NORMAL_TAU = 0 # value of tau to dtermine outliers based on the rule Q1 - tau * IQR =< dist <= Q3 + tau * IQR 
TARGETED_TAU = 1

taus = np.empty(6)
taus[0]=1.5
for i in range(1, 6):
    taus[i] = taus[i-1]-0.4

NOISE_MEAN = 0
NOISE_SD = 0.01
MAXIMUM_ATTACKS = [0, 1, 2, 3, 4, 5, 6]
MAXIMUM_ATTACKS.reverse()
                     
GLOBAL_ROUNDS = 30 #"number of rounds of federated model training"
LOCAL_EPOCHS = 5#"the number of local epochs: E for each peer"
LOCAL_BS = 20 #"local batch size: B for each peer"
LOCAL_LR =  0.001#local learning rate: lr for each peer
LOCAL_MOMENTUM = 0.9 #local momentum for each peer
NUM_CLASSES = 10 # number of classes in an experiment

LABELS_DICT = {"zero":0, 
        "one":1,
        "two":2,
        "three":3,
        "four":4,
        "five":5,
        "six":6,
        "seven":7,
        "eight":8,
        "nine":9}

CLASSES_LIST = ('Zero', 'One', 'Two',  'Three',  'Four',  'Five',
                       'Six',   'Seven', 'Eight', 'Nine')

#select the device to work with cpu or gpu
if torch.cuda.is_available():
    DEVICE = "cuda"
else:
    DEVICE = "cpu"
    
DEVICE = torch.device(DEVICE)

random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)
np.random.seed(SEED)
os.environ['PYTHONHASHSEED'] = str(SEED)

In [ ]:
#Normal training experiment in a simple environment 100% honest peers
normal_training_exp(sim_num=1, peer_pseudonym = PEER_PSEUDONYM,
                peer_num=PEER_NUM, peer_frac = PEER_FRAC, seed = SEED,tau =NORMAL_TAU,
               global_rounds = GLOBAL_ROUNDS, local_epochs = LOCAL_EPOCHS, local_bs = LOCAL_BS,
               local_lr = LOCAL_LR, local_momentum = LOCAL_MOMENTUM, num_classes = NUM_CLASSES, 
               labels_dict = LABELS_DICT, classes_list = CLASSES_LIST, device = DEVICE)

In [ ]:
#Random attack training experiment in a simple environment (Under Random Attack)

untargeted_attack_training_exp(sim_num = 1, peer_pseudonym = PEER_PSEUDONYM,
                peer_num = PEER_NUM, peer_frac = PEER_FRAC, seed = SEED, tau = NORMAL_TAU,
               global_rounds = GLOBAL_ROUNDS, local_epochs = LOCAL_EPOCHS, local_bs = LOCAL_BS,
               local_lr = LOCAL_LR, local_momentum = LOCAL_MOMENTUM, num_classes = NUM_CLASSES, 
               labels_dict = LABELS_DICT, classes_list = CLASSES_LIST, device = DEVICE,
                               attack_type ='untargeted', attack_rates = UNTARGETED_ATTACK_RATES, 
                              malicious_behaviour_rate = RANDOM_MALICIOU_BEHAVIOUR_RATE)

In [ ]:
#Random attack training experiment in a secure environment (Under Random Attack)

secureEnv_random_attack_training_exp(sim_num = 1, peer_pseudonym = PEER_PSEUDONYM,
                peer_num = PEER_NUM, peer_frac = PEER_FRAC, seed = SEED, tau = NORMAL_TAU, 
                global_rounds = GLOBAL_ROUNDS, local_epochs = LOCAL_EPOCHS, local_bs = LOCAL_BS,
               local_lr = LOCAL_LR , local_momentum = LOCAL_MOMENTUM, num_classes = NUM_CLASSES, 
               labels_dict = LABELS_DICT, classes_list = CLASSES_LIST, device = DEVICE,
                attack_type = 'untargeted', attack_rates = UNTARGETED_ATTACK_RATES, mapping_list = MAPPING_LIST, 
                                          malicious_behaviour_rate = RANDOM_MALICIOU_BEHAVIOUR_RATE)

In [ ]:
# Targeted attack training experiment in a simple environment (Under Targeted Attack with no detection)

for m in MAXIMUM_ATTACKS:
    
    print('Maximum number of attacks: ', m)
    
    targeted_attack_training_exp(sim_num = 1, peer_pseudonym = PEER_PSEUDONYM,
                    peer_num = PEER_NUM, peer_frac = PEER_FRAC, seed = SEED,tau = TARGETED_TAU,
                   global_rounds = GLOBAL_ROUNDS, local_epochs = LOCAL_EPOCHS, local_bs = LOCAL_BS,
                   local_lr = LOCAL_LR, local_momentum = LOCAL_MOMENTUM, num_classes = NUM_CLASSES, 
                   labels_dict = LABELS_DICT, classes_list = CLASSES_LIST, device = DEVICE,
                                   attack_type ='targeted', attack_rates = STEALTHY_POSONING_RATES, 
                                 malicious_behaviour_rate = STEALTHY_MALICIOU_BEHAVIOUR_RATE, mapping_list = MAPPING_LIST,
                                maximum_attacks=m)

In [3]:
# Targeted attack training experiment in a secure environment (Under Targeted Attack with no detection)

for m in MAXIMUM_ATTACKS:
    
    print('Maximum number of attacks: ', m)
    print('Tau value : ', taus[m-1])
    
    secureEnv_targeted_attack_training_exp(sim_num = 1, peer_pseudonym = PEER_PSEUDONYM,
                    peer_num = PEER_NUM, peer_frac = PEER_FRAC, seed = SEED,tau = taus[m-1],
                   global_rounds = GLOBAL_ROUNDS, local_epochs = LOCAL_EPOCHS, local_bs = LOCAL_BS,
                   local_lr = LOCAL_LR, local_momentum = LOCAL_MOMENTUM, num_classes = NUM_CLASSES, 
                   labels_dict = LABELS_DICT, classes_list = CLASSES_LIST, device = DEVICE,
                                   attack_type ='targeted', attack_rates = STEALTHY_POSONING_RATES, 
                                 malicious_behaviour_rate = STEALTHY_MALICIOU_BEHAVIOUR_RATE, mapping_list = MAPPING_LIST,
                                maximum_attacks=m)

Maximum number of attacks:  6
Tau value :  -0.5

==>Start targeted attack Simulation in Secure Environment...


===>targeted attack with rate of: (40%) of peers and malicious behaviour rate of: (100%)
======>Creating Global model.....
CNNMnist(
  (conv1): Conv2d(1, 10, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(10, 20, kernel_size=(5, 5), stride=(1, 1))
  (conv2_drop): Dropout2d(p=0.5, inplace=False)
  (fc1): Linear(in_features=320, out_features=50, bias=True)
  (fc2): Linear(in_features=50, out_features=10, bias=True)
)

===>Simulation Step: 1

Loading done!....


=======> Start Global Model Training......




Selected workers

[32 93 79 54 20 23 66 25 36 75 33 62 95 68 49]
Worker 32 : is  Honest
Worker 93 : is  Honest
Worker 79 : is  Stealthy_Attacker
Worker 54 : is  Honest
Worker 20 : is  Honest
Worker 23 : is  Stealthy_Attacker
Worker 66 : is  Stealthy_Attacker
Worker 25 : is  Honest
Worker 36 : is  Honest
Worker 75 : is  Stealthy_Attacker
Worker 33 : is  Stealthy_Attacker
Worker 62 : is  Honest
Worker 95 : is  Honest
Worker 68 : is  Honest
Worker 49 : is  Honest

 | Global Training Round : 1 |


Worker 32 starts training in Global Round:1 |


Train Epoch: 1 [0/596	(1%)]	Loss: 2.324178
Train Epoch: 2 [0/596	(1%)]	Loss: 2.301140
Train Epoch: 3 [0/596	(1%)]	Loss: 2.242644
Train Epoch: 4 [0/596	(1%)]	Loss: 2.259973
Train Epoch: 5 [0/596	(1%)]	Loss: 2.236674

Worker 32 ends training in Global Round:1 |

Worker 93 starts training in Global Round:1 |


Train Epoch: 1 [0/596	(1%)]	Loss: 2.329423
Train Epoch: 2 [0/596	(1%)]	Loss: 2.301955
Train Epoch: 3 [0/596	(1%)]	Loss: 2.314887
Train Epoch: 4 [0/596	(1%)]	Loss: 2.254878
Train Epoch: 5 [0/596	(1%)]	Loss: 2.257310

Worker 93 ends training in Global Round:1 |

Worker 79 starts training in Global Round:1 |
Performed attacks 0

===>Targeted/stealthy attack started by:  Worker 79  in Global Round:  1


Train Epoch: 1 [0/596	(1%)]	Loss: 2.281091
Train Epoch: 2 [0/596	(1%)]	Loss: 2.262718
Train Epoch: 3 [0/596	(1%)]	Loss: 2.265451
Train Epoch: 4 [0/596	(1%)]	Loss: 2.214187
Train Epoch: 5 [0/596	(1%)]	Loss: 2.108091

Performed attacks 1
Worker 79 ends training in Global Round:1 |

Worker 54 starts training in Global Round:1 |


...

Train Epoch: 1 [0/596	(1%)]	Loss: 2.333549
Train Epoch: 2 [0/596	(1%)]	Loss: 2.320938
Train Epoch: 3 [0/596	(1%)]	Loss: 2.272410
Train Epoch: 4 [0/596	(1%)]	Loss: 2.232969
Train Epoch: 5 [0/596	(1%)]	Loss: 2.224079

Worker 62 ends training in Global Round:1 |

Worker 95 starts training in Global Round:1 |


Train Epoch: 1 [0/596	(1%)]	Loss: 2.324778
Train Epoch: 2 [0/596	(1%)]	Loss: 2.213844
Train Epoch: 3 [0/596	(1%)]	Loss: 2.317612
Train Epoch: 4 [0/596	(1%)]	Loss: 2.273321
Train Epoch: 5 [0/596	(1%)]	Loss: 2.231489

Worker 95 ends training in Global Round:1 |

Worker 68 starts training in Global Round:1 |


Train Epoch: 1 [0/596	(1%)]	Loss: 2.287653
Train Epoch: 2 [0/596	(1%)]	Loss: 2.279771
Train Epoch: 3 [0/596	(1%)]	Loss: 2.295766
Train Epoch: 4 [0/596	(1%)]	Loss: 2.234906
Train Epoch: 5 [0/596	(1%)]	Loss: 2.248201

Worker 68 ends training in Global Round:1 |

Worker 49 starts training in Global Round:1 |


Train Epoch: 1 [0/596	(1%)]	Loss: 2.305769
Train Epoch: 2 [0/596	(1%)]	Loss: 2.327223
Train Epoch: 3 [0/596	(1%)]	Loss: 2.314443
Train Epoch: 4 [0/596	(1%)]	Loss: 2.348235
Train Epoch: 5 [0/596	(1%)]	Loss: 2.241146

Worker 49 ends training in Global Round:1 |
Last layer biases  [array([-0.01085503,  0.12526898,  0.08547497,  0.12338903,  0.02273891,
       -0.00384581,  0.13120633, -0.07331362, -0.13768356,  0.09034205],
      dtype=float32), array([-0.0144955 ,  0.12565412,  0.08650556,  0.12444799,  0.02103342,
        0.00259337,  0.13220383, -0.07790712, -0.1380766 ,  0.09076324],
      dtype=float32), array([-8.1699118e-03,  1.3440110e-01,  1.0101857e-01,  1.2604679e-01,
        9.1169208e-02, -1.7296239e-03,  1.3840178e-01, -8.1115797e-02,
       -1.4729479e-01, -5.0276476e-06], dtype=float32), array([-0.01443019,  0.12190548,  0.0902397 ,  0.12635401,  0.0236273 ,
       -0.00035232,  0.1326912 , -0.08000032, -0.1365382 ,  0.08922584],
      dtype=float32), array([-0.01465937,  

KeyError: '1.0'